# Independence analysis with IBAMR
---

---
## Using more stringent exit criterion
---

In [ ]:
# WARNING: execute this cell if you have computed the numerical solutions
script = os.path.join('ibamr',
                      'iterativeError',
                      'plotForceCoefficientsCompareRTol.py')
%run $script

In [ ]:
Image(os.path.join('ibamr',
                   'iterativeError',
                   'forceCoefficientsCompareRTol.png'))

---
## Using a smaller CFL constraint
---

In [ ]:
# WARNING: execute this cell if you have computed the numerical solutions
script = os.path.join('ibamr',
                      'temporalError',
                      'plotForceCoefficientsCompareCFL.py')
%run $script

In [ ]:
Image(os.path.join('ibamr',
                   'temporalError',
                   'forceCoefficientsCompareCFL.png'))

---
#### References:

* Krishnan, A., Socha, J. J., Vlachos, P. P., & Barba, L. A. (2014). Lift and wakes of flying snakes. Physics of Fluids (1994-present), 26(3), 031901.

---